In [9]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, Flatten
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping

# Configurações gerais
plt.rcParams.update({'figure.max_open_warning': 0})

# Diretório para salvar as figuras
output_dir = 'figuras_cnn'
os.makedirs(output_dir, exist_ok=True)

# 1. Leitura do dataset a partir do arquivo CSV
df = pd.read_csv('pca_reduced_data.csv')

# Verificar a estrutura do DataFrame
print(f"Formato do DataFrame: {df.shape}")
print(f"Primeiras linhas do DataFrame:\n{df.head()}")

# Seleciona apenas as colunas necessárias (colunas pares)
Q_reduced = df.iloc[:, ::2].values.astype(float)  # Supondo que as colunas pares são as reais

# Transpor para ter as características nas colunas
Q_reduced = Q_reduced.T

# Renomear as colunas de Q_reduced
n_features = Q_reduced.shape[1]
Q_reduced_df = pd.DataFrame(Q_reduced, columns=[f'feature_{i+1}' for i in range(n_features)])
print(f"Características selecionadas: {Q_reduced_df.columns.tolist()}")

# 2. Definição da janela deslizante e número de características
window_size = 25  # Tamanho da janela deslizante para criar sequências temporais

# 3. Normalização dos dados
scaler = MinMaxScaler(feature_range=(-1, 1))
Q_reduced_scaled = scaler.fit_transform(Q_reduced_df)

# 4. Preparação das sequências
def create_sequences(data, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i + window_size])
        labels.append(data[i + window_size])
    return np.array(sequences), np.array(labels)

# Criar sequências a partir dos dados normalizados
X, y = create_sequences(Q_reduced_scaled, window_size)
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# 5. Divisão dos dados em conjuntos de treinamento e teste com continuidade temporal
train_ratio = 0.75
split_index = int(len(X) * train_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# 6. Definição do modelo CNN com camadas adicionais e dropout
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(window_size, n_features)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(n_features))  # Saída com o número de características

# 7. Compilação do modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# 8. Definição do EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 9. Treinamento do modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# 10. Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# 11. Reverter a normalização das previsões e valores reais
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# 12. Calcular métricas de erro
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f'MAE Geral: {mae:.4f}, MSE Geral: {mse:.4f}')

# Função para calcular NMSE por característica
def calculate_normalized_mse(actual, predicted):
    num_features = actual.shape[1]
    nmse = np.zeros(num_features)
    for c in range(num_features):
        mse = mean_squared_error(actual[:, c], predicted[:, c])
        variance = np.var(actual[:, c])
        if variance != 0:
            nmse[c] = mse / variance
        else:
            nmse[c] = np.nan
    return nmse

# Calcular NMSE para cada característica
nmse = calculate_normalized_mse(y_test_inv, y_pred_inv)
average_nmse = np.nanmean(nmse)
print(f"Média do NMSE: {average_nmse:.4f}")

# Remover possíveis NaNs de nmse para plotagem
nmse = np.nan_to_num(nmse, nan=0.0)

# 13. Comparar previsões com valores reais em subplots
num_features_to_plot = 4  # Número de características a serem plotadas
if n_features < num_features_to_plot:
    num_features_to_plot = n_features
    print(f"Reduzindo o número de características para plotagem para {num_features_to_plot}")

# Selecionar características específicas ou aleatórias
features_to_plot = list(range(num_features_to_plot))

fig, axs = plt.subplots(num_features_to_plot, 1, figsize=(12, 3 * num_features_to_plot), dpi=400)

# Ajuste para o caso de apenas um subplot
if num_features_to_plot == 1:
    axs = [axs]

for idx, feature_idx in enumerate(features_to_plot):
    axs[idx].plot(y_test_inv[:, feature_idx], label=f'Valor Real (Feature {feature_idx+1})')
    axs[idx].plot(y_pred_inv[:, feature_idx], label=f'Previsão (Feature {feature_idx+1})', linestyle='--')
    axs[idx].set_title(f'Feature {feature_idx+1} - Previsão vs Valor Real')
    axs[idx].set_xlabel('Amostra')
    axs[idx].set_ylabel('Valor')
    axs[idx].legend()
    axs[idx].grid(True)
    # Rotacionar os rótulos do eixo x
    axs[idx].tick_params(axis='x', rotation=45)
    # Reduzir o número de ticks no eixo x
    axs[idx].xaxis.set_major_locator(plt.MaxNLocator(10))
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_features.png'), dpi=400)
plt.close()

# 14. Plotar NMSE por característica
features = np.arange(1, n_features + 1)
plt.figure(figsize=(12, 6), dpi=400)
plt.bar(features, nmse, width=0.6, color='blue')
plt.xlabel('Índice da Característica')
plt.ylabel('NMSE')
plt.title('NMSE por Característica')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'nmse_por_caracteristica.png'), dpi=400)
plt.close()

# 15. Plotar valores de perda de treinamento e validação
plt.figure(figsize=(12, 6), dpi=400)
# Verificar se 'val_loss' está presente no histórico
if 'val_loss' in history.history:
    plt.plot(history.history['loss'], label='Perda de Treinamento')
    plt.plot(history.history['val_loss'], label='Perda de Validação')
else:
    plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.title('Função de Perda ao Longo das Épocas')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'funcao_perda.png'), dpi=400)
plt.close()


Formato do DataFrame: (10, 800)
Primeiras linhas do DataFrame:
   column_0_real  column_0_imag  column_1_real  column_1_imag  column_2_real  \
0     -20.428307            0.0     -14.548930            0.0      -8.572894   
1     -47.603580            0.0     -48.728410            0.0     -49.391586   
2     -11.588748            0.0      -9.705402            0.0      -7.482475   
3      -7.333770            0.0      -9.196754            0.0     -10.665637   
4       0.143841            0.0      -1.604118            0.0      -3.173469   

   column_2_imag  column_3_real  column_3_imag  column_4_real  column_4_imag  \
0            0.0      -2.563102            0.0       3.420115            0.0   
1            0.0     -49.598959            0.0     -49.356401            0.0   
2            0.0      -5.010855            0.0      -2.386676            0.0   
3            0.0     -11.701604            0.0     -12.281145            0.0   
4            0.0      -4.430638            0.0      -5.2

c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.4480 - val_loss: 0.1709
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1558 - val_loss: 0.0592
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0592 - val_loss: 0.0193
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0338 - val_loss: 0.0099
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0250 - val_loss: 0.0058
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0192 - val_loss: 0.0061
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0165 - val_loss: 0.0038
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0150 - val_loss: 0.0032
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0139 - val_loss: 0.0041
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0125 - val_loss: 0.0023
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0122 - val_loss: 0.0017
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0110 - val_loss: 0.0019
Epoch 13/1